In [1]:
#pip install tensorflow

In [2]:
import os
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
directory = "TImages/"

In [4]:
image_size = (244, 244)
classes = ["0", "1", "2"]

In [5]:
# Create an ImageDataGenerator instance with data augmentation settings
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    samplewise_center=True,
    samplewise_std_normalization=True,
)

In [6]:
training_data = []

def create_training_data():
    for category in classes:
        path = os.path.join(directory, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, image_size)

                # Generate and store augmented images
                augmented_images = []
                augmented_images.append(new_array)  # Original image
                img_array_aug = new_array.reshape((1,) + new_array.shape)
                i = 0
                for batch in datagen.flow(img_array_aug, batch_size=1):
                    augmented_images.append(batch[0])
                    i += 1
                    if i >= 5:  # Generate 5 augmented images per input image
                        break

                for augmented_image in augmented_images:
                    image_hsv = cv2.cvtColor(augmented_image, cv2.COLOR_BGR2HSV)
                    training_data.append([image_hsv, class_num])

            except Exception as e:
                pass

In [7]:
create_training_data()

In [8]:
lenofimage = len(training_data)
print(lenofimage)

5850


In [9]:
X=[]
y=[]

for categories, label in training_data:
    X.append(categories)
    y.append(label)
X= np.array(X).reshape(lenofimage,-1)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, )

In [12]:
## GB MODEL

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [14]:
# Feature Scaling
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
# Hyperparameter Tuning using GridSearchCV
param_grid = {'n_estimators': [50, 100],
              'learning_rate': [0.01, 0.1],
              'max_depth': [3, 5],
              'max_features': ['auto', 'log2'],
              'loss': ['log_loss'],
              'subsample': [0.5, 1],
             
             }

In [16]:
gb = GradientBoostingClassifier()

In [17]:
# Create the GridSearch object without cross-validation
grid_search = GridSearchCV(gb, param_grid, cv=5, verbose=2)  # Set cv=None for no cross-validation

In [18]:
# Fit the GridSearch object to your data
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=auto, n_estimators=50, subsample=0.5; total time=   0.7s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=auto, n_estimators=50, subsample=0.5; total time=   1.1s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=auto, n_estimators=50, subsample=0.5; total time=   0.7s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=auto, n_estimators=50, subsample=0.5; total time=   0.7s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=auto, n_estimators=50, subsample=0.5; total time=   1.0s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=auto, n_estimators=50, subsample=1; total time=   0.7s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=auto, n_estimators=50, subsample=1; total time=   0.7s
[CV] END learning_rate=0.01, loss=log_loss, max_depth

[CV] END learning_rate=0.01, loss=log_loss, max_depth=5, max_features=log2, n_estimators=50, subsample=1; total time=   7.7s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=5, max_features=log2, n_estimators=50, subsample=1; total time=   7.7s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=5, max_features=log2, n_estimators=50, subsample=1; total time=   7.8s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=5, max_features=log2, n_estimators=50, subsample=1; total time=   7.8s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=5, max_features=log2, n_estimators=50, subsample=1; total time=   7.8s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=5, max_features=log2, n_estimators=100, subsample=0.5; total time=   8.2s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=5, max_features=log2, n_estimators=100, subsample=0.5; total time=   8.1s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=5, max_features=log2, n_estimators=100, subsample=0.5; total time

[CV] END learning_rate=0.1, loss=log_loss, max_depth=5, max_features=auto, n_estimators=100, subsample=0.5; total time=   0.7s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=5, max_features=auto, n_estimators=100, subsample=0.5; total time=   0.6s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=5, max_features=auto, n_estimators=100, subsample=0.5; total time=   0.6s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=5, max_features=auto, n_estimators=100, subsample=0.5; total time=   0.5s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=5, max_features=auto, n_estimators=100, subsample=1; total time=   0.7s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=5, max_features=auto, n_estimators=100, subsample=1; total time=   0.6s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=5, max_features=auto, n_estimators=100, subsample=1; total time=   0.6s
[CV] END learning_rate=0.1, loss=log_loss, max_depth=5, max_features=auto, n_estimators=100, subsample=1; total time=

C:\Users\tasli\anaconda3\envs\coad\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
80 fits failed out of a total of 160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
80 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\tasli\anaconda3\envs\coad\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\tasli\anaconda3\envs\coad\lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "C:\Users\tasli\anaconda3\envs\coad\lib\site-packages\sklearn\base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "C:

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.01, 0.1], 'loss': ['log_loss'],
                         'max_depth': [3, 5], 'max_features': ['auto', 'log2'],
                         'n_estimators': [50, 100], 'subsample': [0.5, 1]},
             verbose=2)

In [19]:
# Get the best estimator and evaluate it
best_nb = grid_search.best_estimator_
y_test_pred = best_nb.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
print("Best Gaussian Naive Bayes Parameters:", grid_search.best_params_)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Best Gaussian Naive Bayes Parameters: {'learning_rate': 0.1, 'loss': 'log_loss', 'max_depth': 5, 'max_features': 'log2', 'n_estimators': 100, 'subsample': 1}
Accuracy: 95.13%


In [20]:
# work progressing...........


In [21]:
grid_search.cv_results_

{'mean_fit_time': array([ 0.92885523,  0.82562628,  0.76280761,  0.77501326,  3.46935534,
         5.14681644,  5.40348649,  8.85015883,  0.73314538,  0.74394655,
         0.73154397,  0.71794233,  4.58670306,  7.44206715,  7.82554197,
        13.42605267,  0.72374225,  0.71313982,  0.7909555 ,  0.72234197,
         3.50089011,  5.23683195,  5.43226509,  8.93676095,  0.76074944,
         0.74394569,  0.72574301,  0.74914722,  4.63431287,  7.47467346,
         7.75472832, 13.43844905]),
 'std_fit_time': array([0.18007436, 0.04238815, 0.04296641, 0.08632073, 0.01992155,
        0.02965092, 0.01765663, 0.04802429, 0.04456973, 0.03653884,
        0.08522727, 0.04284418, 0.0281799 , 0.03730984, 0.01587743,
        0.07018675, 0.04665927, 0.04584724, 0.06519967, 0.04355802,
        0.03038746, 0.01867365, 0.03062091, 0.04783932, 0.09574472,
        0.07967842, 0.04997513, 0.02636425, 0.02801697, 0.04680876,
        0.01913992, 0.07719319]),
 'mean_score_time': array([0.        , 0.        , 

In [22]:
df = pd.DataFrame(grid_search.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_loss,param_max_depth,param_max_features,param_n_estimators,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.928855,0.180074,0.000000,0.000000,0.01,log_loss,3,auto,50,0.5,"{'learning_rate': 0.01, 'loss': 'log_loss', 'm...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,17
1,0.825626,0.042388,0.000000,0.000000,0.01,log_loss,3,auto,50,1,"{'learning_rate': 0.01, 'loss': 'log_loss', 'm...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,17
2,0.762808,0.042966,0.000000,0.000000,0.01,log_loss,3,auto,100,0.5,"{'learning_rate': 0.01, 'loss': 'log_loss', 'm...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,17
3,0.775013,0.086321,0.000000,0.000000,0.01,log_loss,3,auto,100,1,"{'learning_rate': 0.01, 'loss': 'log_loss', 'm...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,17
4,3.469355,0.019922,0.387106,0.001899,0.01,log_loss,3,log2,50,0.5,"{'learning_rate': 0.01, 'loss': 'log_loss', 'm...",0.935897,0.935897,0.935897,0.935897,0.934829,0.935684,0.000427,14
5,5.146816,0.029651,0.386707,0.001020,0.01,log_loss,3,log2,50,1,"{'learning_rate': 0.01, 'loss': 'log_loss', 'm...",0.935897,0.935897,0.935897,0.935897,0.934829,0.935684,0.000427,14
6,5.403486,0.017657,0.397109,0.001415,0.01,log_loss,3,log2,100,0.5,"{'learning_rate': 0.01, 'loss': 'log_loss', 'm...",0.938034,0.936966,0.936966,0.938034,0.934829,0.936966,0.001170,12
7,8.850159,0.048024,0.395502,0.000496,0.01,log_loss,3,log2,100,1,"{'learning_rate': 0.01, 'loss': 'log_loss', 'm...",0.939103,0.936966,0.936966,0.939103,0.935897,0.937607,0.001282,11
8,0.733145,0.044570,0.000000,0.000000,0.01,log_loss,5,auto,50,0.5,"{'learning_rate': 0.01, 'loss': 'log_loss', 'm...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,17
9,0.743947,0.036539,0.000000,0.000000,0.01,log_loss,5,auto,50,1,"{'learning_rate': 0.01, 'loss': 'log_loss', 'm...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,17


In [24]:
df[[ 'params', 'mean_test_score',  'rank_test_score']]

,params,mean_test_score,rank_test_score
0,"{'learning_rate': 0.01, 'loss': 'log_loss', 'm...",NaN,17
1,"{'learning_rate': 0.01, 'loss': 'log_loss', 'm...",NaN,17
2,"{'learning_rate': 0.01, 'loss': 'log_loss', 'm...",NaN,17
3,"{'learning_rate': 0.01, 'loss': 'log_loss', 'm...",NaN,17
4,"{'learning_rate': 0.01, 'loss': 'log_loss', 'm...",0.935684,14
5,"{'learning_rate': 0.01, 'loss': 'log_loss', 'm...",0.935684,14
6,"{'learning_rate': 0.01, 'loss': 'log_loss', 'm...",0.936966,12
7,"{'learning_rate': 0.01, 'loss': 'log_loss', 'm...",0.937607,11
8,"{'learning_rate': 0.01, 'loss': 'log_loss', 'm...",NaN,17
9,"{'learning_rate': 0.01, 'loss': 'log_loss', 'm...",NaN,17


In [25]:
grid_search.best_score_

0.9623931623931623

In [26]:
grid_search.best_params_

{'learning_rate': 0.1,
 'loss': 'log_loss',
 'max_depth': 5,
 'max_features': 'log2',
 'n_estimators': 100,
 'subsample': 1}